### Imports & Env Setup

In [6]:
%reload_ext autoreload
%autoreload 2
import sys
import os
from dotenv import load_dotenv
load_dotenv()

import dspy
sys.path.append(os.path.abspath('../'))
from prompt_migrator.benchmarks import llama_mmlu_pro, leaderboard_mmlu_pro

### Configuration

In [32]:
NUM_THREADS = 16

FEW_SHOTS = 5

# See https://docs.litellm.ai/docs/providers/vllm for details
TASK_MODEL = dspy.LM(
    "hosted_vllm/meta-llama/Llama-3.3-70B-Instruct",
    api_base = 'http://localhost:8000/v1' , # or api_base ?
    # api_version: Optional[str] = None,
    # api_key: Optional[str] = None,
    # seed: Optional[int] = None,
    # max_tokens: Optional[int] = None,
    # timeout: Optional[Union[float, int]] = None,
)
PROMPT_MODEL = dspy.LM(
    "hosted_vllm/meta-llama/Llama-3.3-70B-Instruct",
    api_base = 'http://localhost:8000/v1', # or api_base ?
    # api_version: Optional[str] = None,
    # api_key: Optional[str] = None,
    # seed: Optional[int] = None,
    # max_tokens: Optional[int] = None,
    # timeout: Optional[Union[float, int]] = None,
)

dspy.configure(lm=TASK_MODEL)

# replace this with llama_mmlu_pro or whatever
benchmark = leaderboard_mmlu_pro

# Without chain of thought:
# program = dspy.Predict(
#     benchmark.signature("")
# )

# With chain of thought:
program = dspy.ChainOfThought(
    benchmark.signature("") # put your initial system prompt here, or leave blank
)

evaluate = dspy.Evaluate(
    devset=[],
    metric=benchmark.metric,
    num_threads=NUM_THREADS,
    display_progress=True,
    display_table=True,
    return_all_scores=True,
    return_outputs=True,
)

### Load dataset

In [33]:
trainset, valset, testset = benchmark.datasets(
    train_size=0.1,
    validation_size=0.2,
)

len(trainset), len(valset), len(testset)

(1203, 2165, 8664)

### Optimize Subset + Evaluation

In [34]:
subset_size = 20
optimizer = dspy.MIPROv2(
    metric=benchmark.metric,
    auto="light",
    num_threads=NUM_THREADS,
    task_model=TASK_MODEL,
    prompt_model=PROMPT_MODEL,
    max_labeled_demos=FEW_SHOTS,
)

optimized_program = optimizer.compile(
    program,
    trainset=trainset[:subset_size],
    valset=valset[:subset_size],
    requires_permission_to_run=False,
)

2025/01/15 17:44:49 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 7
minibatch: False
num_candidates: 5
valset size: 20

2025/01/15 17:44:49 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/01/15 17:44:49 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/01/15 17:44:49 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=5 sets of demonstrations...


Bootstrapping set 1/5
Bootstrapping set 2/5
Bootstrapping set 3/5


 20%|███████████████████████████████████████████████                                                                                                                                                                                            | 4/20 [00:20<01:23,  5.19s/it]


Bootstrapped 4 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.
Bootstrapping set 4/5


 40%|██████████████████████████████████████████████████████████████████████████████████████████████                                                                                                                                             | 8/20 [00:52<01:18,  6.56s/it]


...

2025/01/15 17:49:41 INFO dspy.evaluate.evaluate: Average Metric: 14 / 20 (70.0%)
2025/01/15 17:49:41 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 70.0 with parameters ['Predictor 0: Instruction 4', 'Predictor 0: Few-Shot Set 3'].
2025/01/15 17:49:41 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [25.0, 75.0, 80.0, 75.0, 80.0, 70.0]
2025/01/15 17:49:41 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 80.0
2025/01/15 17:49:41 INFO dspy.teleprompt.mipro_optimizer_v2: =======================


2025/01/15 17:49:41 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 6 / 7 =====



Average Metric: 15.00 / 20 (75.0%): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:24<00:00,  1.25s/it]

2025/01/15 17:50:06 INFO dspy.evaluate.evaluate: Average Metric: 15 / 20 (75.0%)
2025/01/15 17:50:06 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 75.0 with parameters ['Predictor 0: Instruction 0', 'Predictor 0: Few-Shot Set 1'].
2025/01/15 17:50:06 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [25.0, 75.0, 80.0, 75.0, 80.0, 70.0, 75.0]
2025/01/15 17:50:06 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 80.0
2025/01/15 17:50:06 INFO dspy.teleprompt.mipro_optimizer_v2: =======================


2025/01/15 17:50:06 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 7 / 7 =====



Average Metric: 15.00 / 20 (75.0%): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:26<00:00,  1.34s/it]

2025/01/15 17:50:33 INFO dspy.evaluate.evaluate: Average Metric: 15 / 20 (75.0%)
2025/01/15 17:50:33 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 75.0 with parameters ['Predictor 0: Instruction 4', 'Predictor 0: Few-Shot Set 4'].
2025/01/15 17:50:33 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [25.0, 75.0, 80.0, 75.0, 80.0, 70.0, 75.0, 75.0]
2025/01/15 17:50:33 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 80.0
2025/01/15 17:50:33 INFO dspy.teleprompt.mipro_optimizer_v2: =======================


2025/01/15 17:50:33 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 80.0!


In [35]:
print("BEST PROMPT:\n", optimized_program.predict.signature.instructions)

BEST PROMPT:
 To answer multiple-choice questions that require reasoning and analysis of the subject matter, follow these steps: 

1. Read the question carefully and identify the key concepts and information provided.
2. Analyze the options and determine which ones are plausible based on the information provided in the question.
3. Use a combination of natural language processing and knowledge retrieval to generate a step-by-step reasoning process to arrive at an answer.
4. Evaluate the reasoning process and select the most appropriate answer based on the analysis.
5. Provide a clear and concise explanation of the reasoning process used to arrive at the answer.

Given the fields `question`, `options`, produce the fields `reasoning`, `answer` by following the above steps and using a language model to generate a step-by-step explanation of how to arrive at the answer.


In [36]:
score, results, all_scores = evaluate(
    optimized_program,
    devset=testset[:subset_size],
)

Average Metric: 13.00 / 20 (65.0%): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:26<00:00,  1.32s/it]

2025/01/15 17:52:03 INFO dspy.evaluate.evaluate: Average Metric: 13 / 20 (65.0%)


,question,options,example_answer,reasoning,pred_answer,metric
0,A certain map uses a scale of 1 inch equals 25 miles. How many mil...,"[A. 50, B. 150, C. 100, D. 25, E. 5, F. 200, G. 75, H. 125, I. 175...",H,"To find the number of miles represented by 5 inches on the map, we...",B,
1,When a ball at rest hangs by a single vertical string tension in t...,"['A. may be equal to mg depending on the speed of the ball', 'B. i...",D,"To solve this problem, we need to consider the forces acting on th...",D,✔️ [True]
2,"A 125-kW, 250-V, 1800-rev/min, cumulative compound d-c generator h...","['A. 90%, 200 hp', 'B. 87.5%, 195 hp', 'C. 84.7%, 175 hp', 'D. 88....",D,To find the efficiency and input horsepower requirements of the cu...,E,
3,The diameter of the objective lens of a telescope is 30 mm and the...,"[A. 80%, B. 70%, C. 100%, D. 30%, E. 40%, F. 90%, G. 25%, H. 60%, ...",B,To determine the fraction of the area of the objective lens that i...,B,✔️ [True]
4,"The temperature at the point $(x, y, z)$ in a substance with condu...","[A. $800\pi$, B. $1504\pi$, C. $1248\pi$, D. $1560\pi$, E. $960\pi...",C,To find the rate of heat flow inward across the cylindrical surfac...,C,✔️ [True]
5,Which of the following must be done when universal screening data ...,['A. The school must invest in more modern educational technology....,G,When universal screening data show that very few students are succ...,G,✔️ [True]
6,"Sulfurylchloride decomposes according to the equation, SO_2CI_2(g)...","['A. 1.49 × 10^-2, .0386', 'B. 7.45 × 10^-3, .0542', 'C. 2.98 × 10...",A,"To find the equilibrium constant K_p, we can use the equation: K_p...",A,✔️ [True]
7,"A number’s prime factors are 2, 5, 7, 13, and 31. Which of the fol...","[A. 10, B. 25, C. 6, D. 8, E. 15, F. 30, G. 20, H. 4]",A,"To find a factor that must be a factor of the number, we need to l...",A,✔️ [True]
8,How many ribs are there in the human body?,"[A. 42, B. 24, C. 18, D. 36, E. 40, F. 28, G. 22, H. 30, I. 20, J....",B,"The human body typically has 12 pairs of ribs, which are attached ...",B,✔️ [True]
9,To what extent have biological agents been weaponized?,['A. Military professionals believe that the majority of biologica...,C,The question asks about the extent to which biological agents have...,A,


## Medium Optimization

In [37]:
subset_size = 500
optimizer = dspy.MIPROv2(
    metric=benchmark.metric,
    auto="medium",
    num_threads=NUM_THREADS,
    task_model=TASK_MODEL,
    prompt_model=PROMPT_MODEL,
    max_labeled_demos=FEW_SHOTS,
)

optimized_program = optimizer.compile(
    program,
    trainset=trainset[:subset_size],
    valset=valset[:subset_size],
    requires_permission_to_run=False,
)

2025/01/15 17:52:08 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 25
minibatch: True
num_candidates: 19
valset size: 300

2025/01/15 17:52:08 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/01/15 17:52:08 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/01/15 17:52:08 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=19 sets of demonstrations...


Bootstrapping set 1/19
Bootstrapping set 2/19
Bootstrapping set 3/19


  0%|▌                                                                                                                                                       | 2/500 [00:09<40:54,  4.93s/it]


KeyboardInterrupt: 

In [ ]:
print("BEST PROMPT:\n", optimized_program.predict.signature.instructions)

In [ ]:
score, results, all_scores = evaluate(
    optimized_program,
    devset=testset[:subset_size],
    display_table=False,
)

## Heavy Optimization

In [ ]:
optimizer = dspy.MIPROv2(
    metric=benchmark.metric,
    auto="heavy",
    num_threads=NUM_THREADS,
    task_model=TASK_MODEL,
    prompt_model=PROMPT_MODEL,
    max_labeled_demos=FEW_SHOTS,
)

optimized_program = optimizer.compile(
    program,
    trainset=trainset,
    valset=valset,
)

In [ ]:
print("BEST PROMPT:\n", optimized_program.predict.signature.instructions)

In [ ]:
score, results, all_scores = evaluate(
    optimized_program,
    devset=testset,
    display_table=False,
)